## Basics of Nerual ODEs or How I Learned the Velocity of a Falling Object

If you recall from your intro physcis class, the velocity of a falling ball at some time with quadratic drag is given by, 

$$ \frac{dv}{dt} = \theta_1 + \theta_2 v^{2} $$

This is simply newtons law stating that the change velocity with respect to time is given by the the the graviational constant (which works to speed up the ball) minus some constant times the square fo the velocity (which works to reduce the speed of the ball). 

Lets say we have some noisy observations, and we want to find the parameters of this ODE that describes the velocity.

Before starting the exercise, you will want to install the `torchdiffeq` package with `pip install torchdiffeq`

In [ ]:
from scipy.integrate import odeint #This will solve our ODEs
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

def dvdt(v, t, thetas): # velocity of the falling baseball from newton
    ### TODO implement the dynamics function provided above given a list of thetas
    ### Hint: The t argument is just present to make the ode solver happy. It is not actually used in the function.
    return ???

times = np.linspace(0, 3, 50) #We have 50 observations from our inital state 
true_thetas = [10, -.2] #The dynamics parameters for our generated data
initial_velocity = 0 #The initial condition of our velocity

out = odeint(dvdt, initial_velocity, times, (true_thetas,)) #Integrate our dynamics function outputting at each time
vel_train = out[:,0]

# Add some noise to give us our true 'observations'
vel_train = vel_train + np.random.randn(vel_train.size)/10.

# Now, let's pretend like we don't know the true values of our thetas.
guess_thetas = [4., -1.] #let's make an inital guess as to what the dynmaics parameters might be.

### This is for plotting later
meshv = np.linspace(0, 9, 27)
mesht = np.linspace(0, 3, 12)
meshv, mesht = np.meshgrid(meshv,mesht)
changet = np.ones(meshv.shape)*5


### TODO implement the ode solver for your initial guess (guess_thetas)
### arguments of odeint are a function, the initial state, the output times, and extra function args (will be similar to before)
### If you're unsure about this, try playing around with the times and initial velocity to see what happens
out = ???
vel_guess = out[:,0]

plt.figure(figsize =(10,7))
plt.plot(times, vel_train, '.', label = 'Data') #plot the data and the inital guess
plt.plot(times, vel_guess, label = 'Initial Guess')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Ball Velocity Data')
plt.legend()

Now we need to train our parameters. The adjoint state derivatives gives the loss gradient at any point in the network. This problem has no explicit dependance on t which simplifies it a bit. 

$$ \frac{da_{\theta}(t)}{dt} = -a_{v}(t)\frac{\partial f(v(t),\theta)}{\partial \theta} $$

$$ a_{\theta}(t) = \frac{dL}{d\theta} $$

We have two parameters so we need to solve two odes. $ \frac{\partial f(v(t),\theta)}{\partial \theta} $ can be derived from our equation for $f = \frac{dv}{dt}$ above.

$$ \frac{da_{\theta 1}(t)}{dt} = -a_{v}(t) $$

$$ \frac{da_{\theta 2}(t)}{dt} = -a_{v}(t)*v(t)^2 $$

But in order to solve this equation we also need 

$$ \frac{da_{v}(t)}{dt} = -a_{v}(t)\frac{\partial f(v(t),t,\theta)}{\partial v(t)} $$

But since our data is fairly closely spaced, we can assume the the adjoint velocity gradient is constant at each observation.

$$ a_{v}(t) = \frac{dL}{dv(t)} = 2*(v -v_{obs})$$


In [ ]:
class fallingvelocity():
    
    def __init__(self, observations, thetas, times):
        
        self.observations = observations #our observation velocity
        self.thetas = thetas  #list of initial params must have opposite signs!
        self.times = times #our observation times
        
        
    def _dvdt(self, v, t): ### velocity of the falling baseball from newton
        ###Hint: v and t are just  arguments to make odeint happy. They are not used in either of the
        ### adjoint functions
        return ???

    def _adjointodetheta1(self, v, t, Lossgrad): #adjoint ODE for the first parameter
        ###TODO implement the adjoint ODE for theta 1
        ###Hint: v and t are just  arguments to make odeint happy. They are not used in either of the
        ### adjoint functions
        return ???            
    
    def _adjointodetheta2(self, v, t, Lossgrad, v_current): #adjoint ODE for the second parameter
        ###TODO implement the adjoint ODE for theta 2
        return ???

    
    def adjoint(self, output):
        
        currentvel = output #initial velocity 
        
        timeback = self.times[::-1] #since we are integrating backwards through the network
        
        gradsum1 = 0. #This will contain the cumulative loss with respect to parameter 1
        
        gradsum2 = 0. #This will contain the cumulative loss with respect to parameter 2
    
        for i in range(len(self.times)-1): #Solve odes between each datapoint
            
            initial1 = 0.0 #initial condition for the adjoint weights
            initial2 = 0.0 #initial condition for the adjoint weights
            
            observed_velocity = self.observations[-(i+1)]
            # TODO: Calculate loss with respect to current state.
            Lossgrad = ??? #Loss gradient to ODEs at initalize at given observation
            
            ## Here is where we integrate backwards through the network
            
            backward_time_range = timeback[i:(i+2)]
            ###TODO add ajoint for theta1 
            backward = odeint(???)
            gradsum1 += backward[-1] #add this to your loss gradient wrt parameter 1 to get cumulative gradient

            ## TODO Same for parameter 2
            backward2 = odeint(???)
            gradsum2 += backward2[-1] #add this to your loss gradient wrt parameter 2 to get cumulative gradient
            
            #TODO Integrate velocity backward to to get value at next time step
            currentvel = odeint(???)[-1]
              
            
        return gradsum1, gradsum2 #??? What am I returning here?
    
    def forward(self):
        
        #Forward pass integration to get final velocity
        out = odeint(self._dvdt, 0, self.times)
        
        out = out[:,0]
        
        return out #all velocities are stored merely for ease of plotting purposes. 
    
model = fallingvelocity(vel_train, [4., -1.], times)

eps = 0.0005 # Step size to take. Breaks pretty easily, be careful here

for i in range(6500):

    output = model.forward()
    
    gradt1, gradt2 = model.adjoint(output[-1]) ### Only feed the model the final state for backward pass
    
    ### TODO update thetas
    model.thetas[0] -= ???
    model.thetas[1] -= ???
       
    if i % 100 == 0:
        clear_output(wait=True)
        plt.figure(figsize = (15, 7))
        plt.subplot(121)
        plt.plot(times, vel_train, '.r', label='Observations')
        plt.plot(times, output, 'b', label='Leared Dynamics', linewidth = 2)
        plt.xlabel('Time (s)')
        plt.ylabel('Velocity (m/s)')
        plt.legend()
        plt.subplot(122)
        changev = dvdt(meshv, mesht, model.thetas)
        plt.quiver(mesht, meshv, changet, changev, width = .005)
        plt.xlabel('Time (s)')
        plt.ylabel('Velocity (m/s)')
        plt.title('Underlying Dynamics')
        plt.show()
        print('Iter {:04d}'.format(i))

### Question 1: $\frac{dL}{d\theta}$ was reset to zero at each observation point during the backward pass. Why was this?

### Question 2: What would happen the velocity trajectory if we only had observations of the initial and final state?

### Question 3: This model makes the assumption that $\frac{dL}{dv}$ is constant between observations. Apparently it did not affect the model's ability to converge. When might this assumption cause problems?

### Question 4: Run the model again, but alter the observation data times so they are no longer evenly spaced. Does this method still work?

## Let's find the velocity again with a neural network

In the previous example, we explicitly knew the ODE that described the dynamics of the velocity.

$$ \frac{dv}{dt} = \theta_1 + \theta_2 v^{2} $$

If we don't know the ODE that describes the velocity, we can represent it with a neural network!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint as odeint_nn
from torchdiffeq import odeint_adjoint ## Can use this if we want to use the adjoint method

# NN that will be used to approximate dvdt 
class ODEFunc(nn.Module):

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

    
# Number of time steps to integrate forward
batch_time = 10
# Number of batch samples to use in each gradient descent step
batch_size = 10

# This is how minibatch is imiplemented in the torchdiffeq examples. Interestingly, the model can get good 
# results by only training on 10 data points at a time.
def get_batch(y, t):
    
    true_y = torch.Tensor(y)
    t = torch.Tensor(t)
    
    s = torch.from_numpy(np.random.choice(np.arange(len(t) - batch_time, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y

In [ ]:
vel_train2 = vel_train[:,np.newaxis]

model = ODEFunc()
optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

for itr in range(1000):
    
    optimizer.zero_grad()
    
    # Get initial state, time steps, true y values at each timestep.
    batch_y0, batch_t, batch_y = get_batch(vel_train2, times)

    # TODO: Use NODE to integrate forward
    # Hint: The odeint implemented in the torchdiffeq package uses the same syntax as the scipy odeint function.
    #       Only now, we are using a NN instead of a predefined dvdt function.
    pred_y = odeint_nn(???)
    # NOTE: We are just backpropagating throught the ODE solver here. However, if we want to use the 
    #       adjoint method implemented in torchdiffeq, all we have to do is call odeint_adjoint() rather
    #       than odeint_nn(). Both functions use the same syntax.
    
    
    # loss = torch.sqrt(torch.mean((pred_y - batch_y) ** 2))
    loss = torch.mean(torch.abs(pred_y - batch_y))
    
    loss.backward()
    optimizer.step()

    if itr % 100 == 0:
        clear_output(wait=True)
        with torch.no_grad():
            plt.figure(figsize = (15, 7))
            plt.subplot(121)
            # TODO: Run model forward from time zero at all time steps so we can track progress
            pred_y = odeint_nn(???)
            loss = torch.mean(torch.abs(pred_y - torch.Tensor(vel_train2)))
            plt.plot(times, vel_train2, '.r', label='Observations')
            plt.plot(times, pred_y, 'b', label='NODE Model')
            plt.xlabel('Time (s)')
            plt.ylabel('Velocity (m/s)')
            plt.legend()
            plt.subplot(122)
            
            for i in range(4*3):
                for j in range(9*3):
                    changev = model.forward(i/3, torch.tensor([float(j/3)]))
                    plt.quiver(i/3, j/3, 5, changev, width = .005, headwidth = 3)
            plt.xlabel('Time (s)'),
            plt.ylabel('Velocity (m/s)')
            plt.title('Underlying Dynamics')
            plt.show()
            print('Iter {:04d} | Loss {:.6f}'.format(itr, loss.item()))
            

###  Question 5: Try swapping odeint_nn with odeint_adjoint and retraining the model. Is there a change in runtime or number of iterations before convergence? Why do you think we are seeing these differences between the two methods?

### Question 6: Why might this example be a bad problem to solve with the adjoint method?
